In [ ]:
# clarify names

data = data.rename(columns={'lag': 'news_horizon'})

In [ ]:
d = './dataset.csv'
data = pandas.read_csv(d)
data = data.rename(columns={'lag': 'news_horizon'})

tsi_names = ['news_time']
y_names = ['open_LAG0']
removes = ['ticker', 'time', 'id', 'title', 'news_time']

In [ ]:
def load_data(data, tsi_names, y_names, removes):

    # clarify names

    exclude = y_names + removes
    x_names = [x for x in data.columns.values if x not in exclude]
    print(x_names)
    
    # sample (without folds)

    kind = 'nofolds'
    test_rate = 0.2
    T_, X_, Y_ = numpy.array(tsi_names), numpy.array(x_names), numpy.array(y_names)
    T, X, Y = data[T_].values, data[X_].values, data[Y_].values
    T_train, X_train, Y_train, T_test, X_test, Y_test = ts_sampler(T, X, Y, None, test_rate, kind)
    
    return X_train, Y_train, X_test, Y_test, X_
    

In [ ]:
X_train, Y_train, X_test, Y_test, X_ = load_data(data, tsi_names, y_names, removes)

In [ ]:
# sample (without folds)

kind = 'nofolds'
test_rate = 0.2
T_, X_, Y_ = numpy.array(tsi_names), numpy.array(x_names), numpy.array(y_names)
T, X, Y = data[T_].values, data[X_].values, data[Y_].values
T_train, X_train, Y_train, T_test, X_test, Y_test = ts_sampler(T, X, Y, None, test_rate, kind)

In [ ]:
#

In [ ]:
# Let's discover

In [ ]:
# 

In [ ]:
series = pandas.DataFrame(data={'series': Y_train.ravel()})

In [ ]:
series.hist(bins=20)
series['series'].value_counts()

In [ ]:
series.hist(bins=20)
series['series'].value_counts()

In [ ]:
series.query("-0.003 <= series <= 0.003").hist(bins=20)
series.query("-0.003 <= series <= 0.003")['series'].value_counts()

In [ ]:
series.query("series >= 0.003").hist(bins=20)
series.query("series >= 0.003")['series'].value_counts()

In [ ]:
series.query("series <= -0.003").hist(bins=20)
series.query("series <= -0.003")['series'].value_counts()

In [ ]:
#

In [ ]:
ols_model_sklearn = LinearRegression(fit_intercept=True, n_jobs=-1)
ols_model_sklearn.fit(X=X_train, y=Y_train)

In [ ]:
summary = get_ols_summary(ols_model_sklearn, X_train, Y_train, X_)
Y_hat_train = ols_model_sklearn.predict(X_train)
Y_hat_test = ols_model_sklearn.predict(X_test)
print("{0:.4f}".format(r2_adj(Y_train, Y_hat_train, X_train.shape[0], X_train.shape[1])))
print("{0:.4f}".format(r2_adj(Y_train, Y_hat_train, X_train.shape[0], X_train.shape[1])))
summary

In [ ]:
# shall we keep intercept?

In [ ]:
ols_model_sklearn_params = {...}

In [ ]:
# Feature Selection

In [ ]:
def censory(model, XX, YY):
    if XX.shape[1] == 0:
        mask = numpy.array([False])
    else:
        p_values = get_ols_summary(model, XX, YY, list(range(XX.shape[1])))['Probabilities'].values[1:]
        mask = p_values < 0.05
    return mask

In [ ]:
X_train_adj, Y_train_adj, X_test_adj, Y_test_adj, X_adj_ = recursive_batch(ols_model_sklearn, ols_model_sklearn_params, X_train, Y_train, censory, X_test, Y_test, X_)

In [ ]:
ols_model_sklearn_adj = LinearRegression(**ols_model_sklearn_params)
ols_model_sklearn_adj.fit(X=X_train_adj, y=Y_train_adj)

In [ ]:
summary = get_ols_summary(ols_model_sklearn_adj, X_train_adj, Y_train_adj, X_adj_)
Y_hat_train_adj = ols_model_sklearn_adj.predict(X_train_adj)
Y_hat_test_adj = ols_model_sklearn_adj.predict(X_test_adj)
print("{0:.4f}".format(r2_adj(Y_train_adj, Y_hat_train_adj, X_train_adj.shape[0], X_train_adj.shape[1])))
print("{0:.4f}".format(r2_adj(Y_train_adj, Y_hat_train_adj, X_train_adj.shape[0], X_train_adj.shape[1])))
summary

In [ ]:
visualise_predictions(ols_model_sklearn_adj, X_train_adj, Y_train_adj, X_test_adj, Y_test_adj, time_series=True)